In [ ]:
# ファイルをまとめる(aiduyaitiディレクトリの中で)
$ cat 46511_25607.csv 46513_25609.csv 46514_25610.csv > in_train.csv
$ cat 46512_25608.csv > in_text.csv

In [ ]:
# ファイルをまとめる(aiduyaitiディレクトリの中で)
$ cat 43651_17887.csv 43837_23037.csv 46405_26862.csv 47076_40110.csv 43652_17886.csv > in_train.csv
$ cat 43866_18752.csv 47074_40109.csv > in_test.csv

In [7]:
!touch out.csv

In [6]:
#  名詞を取り出す関数
import re
import numpy as np
import pyprind
import pandas as pd
import os
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

def createNone(in_name):
    words = ""
    with open("out.csv") as f:
        lines = f.readlines()
    with open("out.csv", "w") as f_out:
        for line in lines:
            f_out.write("{}\n".format(line[:-1]))
        with open("{}".format(in_name)) as f_in:        
            line = f_in.readline()
            row_line = re.match("[^,]*,([^,]*)", line)
            row = row_line.group(1)
            while line:
                search_line = re.match("([^,]*,){3}([^,]*),([^,]*)", line)
                if search_line is not None:
                    if search_line.group(3) == "名詞":
                        words += " " + search_line.group(2)
                line = f_in.readline()
                row_line = re.match("[^,]*,([^,]*)", line)
                if row_line is None:
                    f_out.write("{}\n".format(words))
                    break
#    print(words)
    

In [8]:
# それぞれの名詞を取り出す
createNone("aiduyaiti/in_train.csv")
createNone("akita/in_train.csv")

In [9]:
# 単語を特徴ベクトルに変換
count = CountVectorizer()
with open("out.csv") as f:
    lines = f.readlines()
    bag = count.fit_transform(lines)
    
print(count.vocabulary_)

{'人間': 353, 'やう': 175, 'つて': 120, 'こと': 62, '根気': 1016, 'それ': 101, 'やつ': 176, '城壁': 606, '祭壇': 1257, '神像': 1253, '殿堂': 1062, 'いつ': 23, 'この世': 63, 'ため': 110, '東洋': 1006, '西洋': 1400, '何処': 390, 'まま': 163, 'びた': 152, '腐肉': 1339, '素質': 1279, '遺族': 1510, '友人': 518, 'もの': 174, 'せつ': 91, 'かく': 41, '粉々': 1270, '地中': 598, '四角': 581, '御参り': 800, '故人': 918, '貞女': 1448, '孝子': 675, '貧乏': 1450, 'こんな': 67, '孝行': 676, 'むづかしい': 170, '建物': 774, '手わざ': 884, '時代': 964, 'ところ': 126, '一つ': 263, '風霜': 1593, '曝露': 973, '平気': 757, '野火': 1525, '火事': 1128, '崩壊': 745, '立派': 1267, '石碑': 1241, '世間': 322, 'うち': 32, '土台': 590, '石垣': 1239, '下積み': 311, '追慕': 1487, '研究': 1243, 'たち': 108, '搨拓': 912, '磨滅': 1248, '村里': 1003, '那全': 1512, '山河': 743, '記念': 1420, '無く': 1132, 'いくら': 18, 'むかし': 168, '学識': 683, '手腕': 889, '情味': 845, '大官': 643, '政府': 917, '懐柔': 871, '平素': 759, '山水': 742, '眺め': 1230, '好き': 663, '在任': 595, '同行': 551, '宇宙': 684, '開闢': 1548, '初め': 476, 'ぶん': 154, '遊び': 1501, 'わけ': 182, 'みんな': 166, '湮滅': 1118, 'ここ': 60,

In [10]:
print(bag.toarray())

[[0 0 0 ... 0 0 1]
 [4 1 1 ... 1 1 0]]


In [11]:
import pyprind
import pandas as pd
import os

# 単語の関連性を評価
np.set_printoptions(precision=2)
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

data_train = tfidf.fit_transform(count.fit_transform(lines)).toarray()
print(data_train)

[[0.   0.   0.   ... 0.   0.   0.01]
 [0.02 0.   0.   ... 0.   0.   0.  ]]


In [12]:
# 学習

from sklearn.ensemble import RandomForestClassifier

label_train = [0, 1]

estimator = RandomForestClassifier()

estimator.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
# 元データを入れて試してみる

label_predict = estimator.predict(data_train)

print(label_predict)

[0 1]


In [ ]:
# 試験用のデータを入れてみる